In [1]:
import os
import pandas as pd
import subprocess

In [2]:
databaseFolder = './'
dirs = os.listdir(databaseFolder)

In [4]:
df = pd.DataFrame(dirs, columns=["index"])
df['threads']=0
df['nrhs']=0
df['time1']=0
df['time1']= df['time1'].astype(float)


In [5]:
engine = '../klu_threaded.o'

timeout = 20

#vecbPath = f"../database/{database.at[0, 'name']}/vecb.mtx"

def runSingle(engine, threads, nrhs):
    try:
        #command = [engine, pathMtx, vecbPath, str(repLoops)]
        command = [engine, str(threads), str(nrhs)]
        #print(command)
        p = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=timeout)
        #print(p.stdout.decode('utf-8'))
        if p.returncode == 0:
            outLines = (p.stdout).decode('utf-8').split('\n')
            #print(outLines)
            time1 = float(outLines[-2].split(':')[1])
            
        else:
            print(p.stderr.decode('utf-8'))
            time1 = -1
            
    except subprocess.TimeoutExpired:
        time1 = -2

    return time1


t1 = runSingle(engine, 1, 1)
print(t1)  

127218.0


In [6]:
loc = 0

for threads in range(1,2):
    for p in range(0,3):
        nrhs = 2**p
        t1 = runSingle(engine, threads, nrhs)
        df.loc[loc,'index'] = loc
        df.loc[loc, 'threads'] = threads
        df.loc[loc, 'nrhs'] = nrhs
        df.loc[loc, 'time1'] = t1
        print(threads, nrhs, t1)
        loc += 1
        

1 1 25564.8
1 2 30522.7
1 4 32325.0


In [7]:
print(df)

   threads  nrhs    time1
0      1.0   1.0  25564.8
1      1.0   2.0  30522.7
2      1.0   4.0  32325.0


In [8]:
df.to_csv('results.csv')